In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

import joblib


In [2]:
# Importing data
df = pd.read_csv('data/labelencoded_data.csv',index_col=0)
df.drop(['id', 'std', 'sta', 'fltid','arr_iata','dep_iata','ac'], axis=1,inplace=True)


In [3]:
# Splitting data
X = df[['arrstn','depstn','std_time','sta_time']]
y = df['target'] 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [4]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


/home/pclinux/spiced/data_science/flight_delays_ML/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE: 128.3257319386446


In [5]:
# Save the model to a file
model_filename = './data/api_test_logistic_regression_model.pkl'
joblib.dump(lr, model_filename)

['./data/api_test_logistic_regression_model.pkl']